In [1]:
import numpy as np
from os.path import join as pjoin

In [2]:
file_path = "/home/nienke/Documents/Research/SS_MTI/External_packages/Test_reflectivity/m0"

# Open the misfit values

In [3]:
xis = np.load(pjoin(file_path,"dxi_dms.npy"))

In [4]:
xis

array([[ 3.66611630e-08],
       [ 6.02327077e-09],
       [ 2.92573077e-08],
       [ 0.00000000e+00],
       [ 1.08286713e-09],
       [-3.23183258e-08],
       [ 1.18443211e-07],
       [ 1.18478738e-07],
       [ 1.67344112e-06],
       [ 1.67390652e-06]])

In [5]:
ms = np.load(pjoin(file_path,"m0s.npy"))

In [6]:
ms

array([[ 1.00000000e+00,  9.99999963e-01],
       [ 0.00000000e+00, -6.02327077e-09],
       [ 0.00000000e+00, -2.92573077e-08],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.08286713e-09],
       [ 0.00000000e+00,  3.23183258e-08],
       [ 3.71447700e+00,  3.71447688e+00],
       [ 3.71447700e+00,  3.71447688e+00],
       [ 1.75719800e+00,  1.75719633e+00],
       [ 1.75719800e+00,  1.75719633e+00]])

In [7]:
xis = np.load(pjoin(file_path,"xis.npy"))

In [8]:
xis

array([8.94650114e-08])